In [1]:
from tight_binding.finite_tight_binding import TightBindingHamiltonian
import numpy as np
import scipy.constants as spc



In [2]:
"""
quick_effective_mass_tests.py
Small consistency checks for your 1‑D effective‑mass Hamiltonians.
"""

import numpy as np
import scipy.constants as spc
import scipy.linalg as la

hbar = spc.hbar      

def create_effective_mass_hamiltonian(m, a):
    minv      = 1.0 / m
    dminvdz   = np.gradient(m, a)

    dAdz   = (np.diag(dminvdz[1:],  +1) - np.diag(dminvdz[:-1], -1)) / (2*a)
    d2Adz2 = (np.diag(minv[1:],   +1) + np.diag(minv[:-1],  -1)
              - 2*np.diag(minv)) / (a**2)
    return -(hbar**2 / 2) * (dAdz + d2Adz2)


def create_regular_hamiltonian(m, a, N):
    d2Adz2 = (np.eye(N, k=+1) + np.eye(N, k=-1) - 2*np.eye(N)) / a**2
    return -(hbar**2 / (2*m)) * d2Adz2


def run_tests():
    N  = 200                 
    L  = 10e-9               
    a  = L / (N + 1)         
    m0 = 0.067 * spc.m_e     

    
    m_const = np.full(N, m0)
    H_eff_const = create_effective_mass_hamiltonian(m_const, a)
    H_reg_const = create_regular_hamiltonian(m0, a, N)

    frob_error = np.linalg.norm(H_eff_const - H_reg_const)
    print(f"‣ Const‐mass Frobenius error ‖H_eff − H_reg‖  = {frob_error:.3e}")

    # compare eigen values
    evals_const, _ = la.eigh(H_reg_const)
    evals_const_ev = evals_const / spc.electron_volt

    E1_exact = (hbar**2 * np.pi**2) / (2*m0*L**2) / spc.electron_volt
    print("‣ First three numeric E_n (eV):", evals_const_ev[:3])
    print("‣ Analytic   E_1 (eV):         ", E1_exact)

    
    z = np.linspace(a, L-a, N)            
    m_profile = m0 * (1 + 0.20*np.sin(2*np.pi*z/L))   
    H_eff_var = create_effective_mass_hamiltonian(m_profile, a)

    
    herm_error = np.linalg.norm(H_eff_var - H_eff_var.T.conj())
    evals_var, _ = la.eigh(H_eff_var)
    print(f"‣ Hermiticity check ‖H − H†‖        = {herm_error:.3e}")
    print("‣ Variable‑mass well: first three E_n (eV):",
          evals_var[:3] / spc.electron_volt)


if __name__ == "__main__":
    run_tests()


‣ Const‐mass Frobenius error ‖H_eff − H_reg‖  = 0.000e+00
‣ First three numeric E_n (eV): [0.05612276 0.22447734 0.5050226 ]
‣ Analytic   E_1 (eV):          0.056123904791772025
‣ Hermiticity check ‖H − H†‖        = 3.402e-18
‣ Variable‑mass well: first three E_n (eV): [-0.50293738 -0.18913334  0.80787206]
